In [63]:
import numpy as np
import pandas as pd
import requests
import json
import feather
import os
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

app_path = '/Users/nmouterde/Documents/projectsPython/steam-game-engine/'

In [64]:
tkn = RegexpTokenizer(r'\w+')
stoplist = stopwords.words('english')

In [65]:
scrap_data = feather.read_dataframe(os.path.join('/Users/nmouterde/Documents/projectsPython/steam-game-engine/data_processed/scrap_df.f'))

In [66]:
scrap_data['cat1'] = scrap_data['name'].str.lower()
scrap_data['cat2'] = (scrap_data['cat'] + ' '
                      +scrap_data['genres']).str.lower()
scrap_data['cat3'] = np.where(scrap_data['f_desc'] == scrap_data['about'],
                              (scrap_data['f_desc'] + ' ' +
                               scrap_data['about']).str.lower(),
                              scrap_data['f_desc'].str.lower())

In [67]:
scrap_data

,id,name,f_desc,s_desc,about,dev,pub,cat,genres,score,nb_rec,cat1,cat2,cat3
0,10,Counter-Strike,Play the world's number 1 online action game. ...,,Play the world's number 1 online action game. ...,Valve,Valve,Multi-player Valve Anti-Cheat enabled,Action,97,93978,counter-strike,multi-player valve anti-cheat enabled action,play the world's number 1 online action game. ...
1,20,Team Fortress Classic,One of the most popular online action games of...,,One of the most popular online action games of...,Valve,Valve,Multi-player Valve Anti-Cheat enabled,Action,53,3022,team fortress classic,multi-player valve anti-cheat enabled action,one of the most popular online action games of...


In [68]:
cols = ['cat1', 'cat2', 'cat3']
for col in cols:
    scrap_data[col] = scrap_data.apply(lambda x: tkn.tokenize(x[col]), axis=1)
    scrap_data[col] = scrap_data.apply(lambda x: [w for w in x[col] 
                                                  if w not in stoplist and len(w)>1],
                                       axis=1) 
    scrap_data[col+'_non_tk'] = scrap_data.apply(lambda x: ' '.join(x[col]), axis=1)

In [69]:
scrap_data

,id,name,f_desc,s_desc,about,dev,pub,cat,genres,score,nb_rec,cat1,cat2,cat3,cat1_non_tk,cat2_non_tk,cat3_non_tk
0,10,Counter-Strike,Play the world's number 1 online action game. ...,,Play the world's number 1 online action game. ...,Valve,Valve,Multi-player Valve Anti-Cheat enabled,Action,97,93978,"[counter, strike]","[multi, player, valve, anti, cheat, enabled, a...","[play, world, number, online, action, game, en...",counter strike,multi player valve anti cheat enabled action,play world number online action game engage in...
1,20,Team Fortress Classic,One of the most popular online action games of...,,One of the most popular online action games of...,Valve,Valve,Multi-player Valve Anti-Cheat enabled,Action,53,3022,"[team, fortress, classic]","[multi, player, valve, anti, cheat, enabled, a...","[one, popular, online, action, games, time, te...",team fortress classic,multi player valve anti cheat enabled action,one popular online action games time team fort...


In [70]:
vect = TfidfVectorizer(sublinear_tf=True, max_df=1.0, analyzer='word', stop_words='english')
X = vect.fit_transform(scrap_data['cat3_non_tk']).toarray()

In [84]:
print len(X[0])
print len(vect.get_feature_names())
print vect.get_feature_names()
print X
print scrap_data.cat3.values

51
51
[u'abilities', u'action', u'affects', u'ally', u'based', u'brand', u'character', u'class', u'classes', u'classic', u'compete', u'complete', u'demolition', u'enemy', u'engage', u'enlisted', u'features', u'fortress', u'game', u'games', u'hostages', u'incredibly', u'items', u'man', u'medic', u'missions', u'modes', u'number', u'online', u'play', u'popular', u'possesses', u'realistic', u'rescue', u'role', u'sites', u'spy', u'strategic', u'style', u'success', u'team', u'teammates', u'teams', u'terrorist', u'time', u'unique', u'variety', u'warfare', u'weapons', u'wildly', u'world']
[[ 0.          0.13241578  0.26229445  0.18610575  0.18610575  0.18610575
   0.          0.          0.          0.          0.          0.18610575
   0.          0.18610575  0.18610575  0.          0.          0.
   0.18662466  0.          0.18610575  0.18610575  0.          0.          0.
   0.18610575  0.          0.18610575  0.13241578  0.13241578  0.13241578
   0.          0.18610575  0.18610575  0.26229